In [1]:

X1 = matrix(ZZ,2,3,[
        [4, 7, 2],
        [2, 4, 6]])

X2 = matrix(ZZ,3,3,[
        [-397, 423, 352],
        [   2,  -3,   1],
        [-146, 156, 128],
])

PolQ.<xQ> = PolynomialRing(QQ)
AQ = matrix(PolQ,3,[
            [xQ + 1,  2,     -6],
            [     1, xQ,     -3],
            [     1,  1, xQ - 4]])
Pol2.<x2> = PolynomialRing(FiniteField(2))
AF2 = matrix(Pol2,3,[
            [x2 + 1,  2,     -6],
            [     1, x2,     -3],
            [     1,  1, x2 - 4]])
Pol3.<x3> = PolynomialRing(FiniteField(3))
AF3 = matrix(Pol3,3,[
            [x3 + 1,  2,     -6],
            [     1, x3,     -3],
            [     1,  1, x3 - 4]])
Pol5.<x5> = PolynomialRing(FiniteField(5))
AF5 = matrix(Pol5,3,[
            [x5 + 1,  2,     -6],
            [     1, x5,     -3],
            [     1,  1, x5 - 4]])

In [55]:
def chercher_pivot(k,A,L,R):
    m = A.nrows()
    n = A.ncols()
    if A[k,k] != 0:
        return
    else:
        for i0 in range(k,m):
            for j0 in range(k,n):
                if A[i0,j0] != 0:
                    A.swap_lines(k,i0)
                    L.swap_lines(k,i0)
                    A.swap_columns(k,j0)
                    R.swap_columns(k,j0)

In [73]:
def annuler_en_bas(k,i,A,L,R):
    (d,s,t) = xgcd(A[k,k],A[i,k])
    if A[i,k]%A[k,k]==0:
        d = A[k,k]
        s = 1
        t = 0
    u = -A[i,k]/d
    v = A[k,k]/d
    Lk = A[k,:]
    LLk = L[k,:]
    Li = A[i,:]
    LLi = L[i,:]
    A[k,:] = s*Lk + t*Li
    L[k,:] = s*LLk + t*LLi
    A[i,:] = u*Lk + v*Li
    L[i,:] = u*LLk + v*LLi

In [54]:
def annuler_a_droite(k,j,A,L,R):
    (d,s,t) = xgcd(A[k,k],A[k,j])
    if A[k,j]%A[k,k]==0:
        d = A[k,k]
        s = 1
        t = 0
    u = -A[k,j]/d
    v = A[k,k]/d
    Ck = A[:,k]
    CRk = R[:,k]
    Cj = A[:,j]
    CRj = R[:,j]
    A[:,k] = s*Ck + t*Cj
    R[:,k] = s*CRk + t*CRj
    A[:,j] = u*Ck + v*Cj
    R[:,j] = u*CRk + v*CRj


In [53]:
def divise_le_reste(k,A,L,R):
    m = A.nrows()
    n = A.ncols()
    for j in range(k+1,n):
        for i in range(k+1,m):
            if A[i,j]%A[k,k]!=0:
                A[:,k] = A[:,k] + A[:,j]
                R[:,k]= R[:,k] + R[:,j]
                return

In [23]:
def coefficients_non_nuls(k,A):
    m = A.nrows()
    n = A.ncols()
    for j in range(k+1,n):
        if A[k,j] != 0:
            return True
    for i in range(k+1,m):
        if A[i,k] != 0:
            return True
    return False

In [114]:
def MySNF(A):
    """
    Forme normale de Smith selon votre code
    """
    t = A.base_ring()
    m = A.nrows()
    n = A.ncols()
    H = copy(A)
    L = identity_matrix(t,m)
    U = identity_matrix(t,n)
    # ECRIVEZ VOTRE CODE ICI
    for k in range(min(m,n)):
        while coefficients_non_nuls(k,H):
            chercher_pivot(k,H,L,U)
            for i in range(k+1,m):
                annuler_en_bas(k,i,H,L,U)
            for j in range(k+1,n):
                annuler_a_droite(k,j,H,L,U)
            divise_le_reste(k,H,L,U)
    assert(H-L*A*U==0)
    return H,L,U

[              1               0               0]
[              0          xQ - 1               0]
[              0               0 xQ^2 - 2*xQ + 1]
[              1               0               0]
[              0          xQ - 1               0]
[              0               0 xQ^2 - 2*xQ + 1]


In [127]:
def sont_associes(a,b):
    """
    Puisqu'on travaille dans des anneaux principaux, il suffit de montrer que a divise b et b divise a
    Renvoie vrai si a et b sont associés
    """
    if b==0 and a==0:
        return True
    
    if b!= 0 and a%b==0:
        if a!=0 and b%a == 0:
            return True
    return False

In [141]:
def meme_SNF(H1,H2):
    """
    Renvoie vrai si les coefficients de la SNF sont tous associés
    Et donc que les coefficients de la SNF sont identiques à un facteur inversible près
    Renvoie faux si les deux SNF ne sont pas équivalentes
    """
    if H1.nrows() != H2.nrows():
        return False
    if H1.ncols() !=  H1.ncols():
        return False
    n = H1.ncols()
    m = H1.nrows()
    for k in range(min(n,m)):
        if not sont_associes(H1[k,k],H2[k,k]):
            return False
    return True

In [142]:
H1, L1, U1 = MySNF(X1)
H2,L2,U2 = MySNF(X2)
HQ, _, _ = MySNF(AQ)
HF2, _, _ = MySNF(AF2)
HF3, _, _ = MySNF(AF3)
HF5, _, _ = MySNF(AF5)
smith_forms = [H1,H2,HQ,HF2,HF3,HF5]
matrices = [X1,X2,AQ,AF2,AF3,AF5]

test = True

for i in range(len(matrices)):
    A = matrices[i]
    H = smith_forms[i]
    smith_sage = A.smith_form()[0]
    if not meme_SNF(H,smith_sage):
        test=False
        break
test
print(X1.smith_form())

([1 0 0]
[0 2 0], [1 0]
[0 1], [  2  -7  17]
[ -1   4 -10]
[  0   0   1])
